In [ ]:
import tweepy
from tweepy import Stream, StreamListener, OAuthHandler
import time
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
import socket
import json

In [ ]:
#api key and api secret are considered oauth handler tokens for tweepy
access_token=''
access_token_secret=''
#Access token and secret are considered set_access_token for tweepy
consumer_key=''
consumer_secret=''

In [ ]:
class TweetsListener(StreamListener):
  # tweet object listens for the tweets
    def __init__(self, csocket):
        self.client_socket = csocket
    def on_data(self, data):
        try:  
            msg = json.loads( data )
            print("new message")
           # if tweet is longer than 140 characters
            if "extended_tweet" in msg:
            # add at the end of each tweet "t_end" 
                self.client_socket\
                    .send(str(msg['extended_tweet']['full_text']+"t_end")\
                    .encode('utf-8'))         
                print(msg['extended_tweet']['full_text'])
            else:
            # add at the end of each tweet "t_end" 
                self.client_socket\
                    .send(str(msg['text']+"t_end")\
                    .encode('utf-8'))
                print(msg['text'])
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    def on_error(self, status):
        print(status)
        return True

In [ ]:
def sendData(client_sock):
    print('start sending data from Twitter to socket')
    #authentication based on the credentials
    auth = tweepy.OAuthHandler(access_token, access_token_secret)
    auth.set_access_token(consumer_key, consumer_secret)
    # start sending data from the Streaming API 
    keywords = ["#"]
    twitter_stream = Stream(auth, TweetsListener(client_sock))
    twitter_stream.filter(track = keywords, languages=["en"])

In [ ]:
s = socket.socket()
host = "localhost"    
port = 9009
s.bind((host, port))
print('socket is ready')
s.listen(4)
s.settimeout(20)
print('socket is listening')
client_sock, addr = s.accept()
print("Received request from: " + str(addr))
sendData(client_sock)